# **Rodrigo_Mesquita_DR2_TP3**
## **Coleta de Dados com Python via APIs e WebScraping [24E3_2]**

#### Notebook disponível em: https://github.com/rodrigo1992-cmyk/Faculdade/tree/main/Notebooks%20Trabalhos%202024H2/DR2_TP3

# Exercício 1:
#### Utilize XPath absoluto com a biblioteca lxml para selecionar:
- O nome do diretor do segundo filme listado;
- O id do penúltimo filme listado, independentemente da quantidade de filmes na lista. 

In [86]:
from lxml import etree

tree = etree.parse(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\TP3.xml', parser=etree.XMLParser())
root = tree.getroot()

diretor = root.xpath('/library/movies/movie[2]/director/text()')
penultimo_id = root.xpath('/library/movies/movie[last()-1]/@id')

print(f'Diretor do diretor segundo filme listado: {diretor[0]}')
print(f'ID do penúltimo filme listado: {penultimo_id[0]}')

Diretor do diretor segundo filme listado: Francis Ford Coppola
ID do penúltimo filme listado: 102


# Exercício 2:
#### Utilize XPath relativo com a biblioteca lxml para selecionar:
- Os títulos dos livros ou filmes da categoria fantasia;
- Os nomes (siglas) das moedas utilizadas nos preços que não são dólares americanos (USD);
- Os preços com valor numérico maior que 15 (independentemente da moeda);
- Todos os títulos que começam com "The".

In [136]:
resp1 = root.xpath('.//*[@category="fantasy"]/title/text()')
resp2 = root.xpath('.//price[@currency!="USD"]/attribute::currency')
resp3 = root.xpath('.//price[number(text()) > 15]/text()')
resp4 = root.xpath('.//title[starts-with(text(), "The")]/text()')

print(f'Livros ou filmes da categoria fantasia: {resp1}')
print(f'Moedas que não são dólares americanos: {resp2}')
print(f'Preços com valor numérico maior que 15: {resp3}')
print(f'Títulos que começam com "The": {resp4}')


Livros ou filmes da categoria fantasia: ["Harry Potter and the Philosopher's Stone", 'The Hobbit']
Moedas que não são dólares americanos: ['NZD', 'GBP', 'BRL']
Preços com valor numérico maior que 15: ['19.99', '15.99', '19.99']
Títulos que começam com "The": ['The Hobbit', 'The Godfather']


# Exercício 3:
#### Utilize (apenas) seletores de CSS com lxml para selecionar:
- Todos os nomes de países dos atletas;
- Todos os nomes de atletas que possuem a letra J (maiúscula);
- Todos os nomes de atletas dos EUA (USA) e da Rússia.

In [141]:
%pip install cssselect

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [206]:
from lxml.cssselect import CSSSelector

fpath = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\TP3.html'
doc = etree.parse(fpath, parser=etree.HTMLParser()).getroot()
doc_paises = doc.cssselect('p.country')

#Resposta 1 - Todos os nomes de países dos atletas
for x in doc_paises:
    print(x.text)

USA
Canada
China
Mexico
Russia


In [207]:
#Resposta 2- Todos os nomes de atletas que possuem a letra J (maiúscula)

doc_nomes = tree.cssselect('.name')
for nome in doc_nomes:
    if "J" in nome.text:
        print(nome.text)

John Doe
Jane Smith


In [209]:
#Resposta 3 - Todos os nomes de atletas dos EUA (USA) e da Rússia.

resposta = []
for atleta in doc.cssselect('.athlete'):
    pais = atleta.cssselect('.country')[0].text
    if pais in ['USA', 'Russia']:
        nome = atleta.cssselect('.name')[0].text
        resposta.append(nome)

print(resposta)

['John Doe', 'Olga Ivanova']


# Exercício 4:

#### Utilize seletores de CSS com BeautifulSoup e algum tratamento em Python que seja necessário para alocar as informações de cada atleta do arquivo HTML em um dataframe do Pandas a ser salvo em arquivo CSV e então responda:

- A média de idades dos atletas;
- O atleta ganhador do Speed Test;
- O atleta ganhador do Endurance Test. 

In [287]:
from bs4 import BeautifulSoup
import pandas as pd

with open(fpath, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

dic = []

atletas = soup.select('.athlete') 
for i in atletas:
    nomes = i.select('.name')[0].text
    pais = i.select('.country')[0].text
    idade = i.select('.age')[0].text
    rank = i.select('.rank')[0].text
    
    #Primeiro localizo o span que contem a identificação do teste, e depois pego o span posterior com o resultado
    speed_test = i.find('span', class_='test-name', string='Speed Test')
    if speed_test: veloc = speed_test.find_next('span', class_='result').text

    endurance_test = i.find('span', class_='test-name', string='Endurance Test')
    if endurance_test: resist = endurance_test.find_next('span', class_='result').text

    #Apendando em um dicionario
    dic.append({
        'nome': nomes,
        'pais': pais,
        'idade': idade,
        'rank': rank,
        'velocidade': veloc,
        'resistencia': resist
        })

#Convertendo para DataFrame e exportando para CSV
df = pd.DataFrame(dic)
df.to_csv(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\atletas.csv', index=False)

df

,nome,pais,idade,rank,velocidade,resistencia
0,John Doe,USA,25,1,95,88
1,Jane Smith,Canada,27,3,87,92
2,Liu Wei,China,24,2,89,91
3,Carlos Mendoza,Mexico,26,5,81,84
4,Olga Ivanova,Russia,28,4,92,86


In [289]:
resp1 = df['idade'].astype(int).mean()
resp2 = df['velocidade'].astype(int).max()
resp3 = df['resistencia'].astype(int).max()

print(f'''
Média de idades dos atletas: {resp1}
Atleta ganhador do Speed Test: {resp2}
Atleta ganhador do Endurance Test: {resp3}
''')


Média de idades dos atletas: 26.0
Atleta ganhador do Speed Test: 95
Atleta ganhador do Endurance Test: 92



: 

# Exercício 5: 

#### Acesse a URL https://www.scrapethissite.com/pages/simple/ e obtenha os dados de todos os países, salvando em um dataframe (sem utilizar o comando `pd.read_html`) com as seguintes colunas:

- Country;
- Capital;
- Population;
- Area.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.scrapethissite.com/pages/simple/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [4]:
#Encontrando as divs dos países
paises = soup.find_all('div', class_='country')

#Iterando em cada pais para armazenar os atributos em uma lista
lista = []
for pais in paises:
    nome = pais.find('h3', class_='country-name').text.strip()
    capital = pais.find('span', class_='country-capital').text.strip()
    populacao = pais.find('span', class_='country-population').text.strip()
    area = pais.find('span', class_='country-area').text.strip()
    
    lista.append([nome, capital, populacao, area])

df = pd.DataFrame(lista, columns=['Pais', 'Capital', 'Populacao', 'Area'])
df.head()

,Pais,Capital,Populacao,Area
0,Andorra,Andorra la Vella,84000,468.0
1,United Arab Emirates,Abu Dhabi,4975593,82880.0
2,Afghanistan,Kabul,29121286,647500.0
3,Antigua and Barbuda,St. John's,86754,443.0
4,Anguilla,The Valley,13254,102.0


# Exercício 6: 

#### Escolha uma notícia qualquer do site https://www.romanews.com.br/ e obtenha o HTML da página via requisição utilizando a biblioteca que preferir (`urllib`, `requests`, etc.), salvando o arquivo no seu diretório. Então, a partir do arquivo HTML da notícia salva, construa códigos em Python utilizando `BeautifulSoup` para obter e salvar em um arquivo JSON, junto à URL da notícia e ao datetime do momento da requisição da página (os objetos datetime devem ter a informação da timezone e ser colocados em isoformat no momento de gerar o JSON):

- O objeto datetime da data e hora da publicação da notícia (dica: é sempre bom checar as tags `meta` dentro do `head` do arquivo.);
- O título da notícia;
- O corpo do texto da notícia;
- O subtítulo da notícia (se houver);
- O autor ou autores da notícia (se houver).

In [5]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import pytz

In [87]:
url = "https://www.romanews.com.br/economia/empresario-do-caso-luso-brasileiro-acumula-processos-0924"

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get(url, headers=headers)
dt_requisição = datetime.now(pytz.utc)


path = r"C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\noticia.html"
with open(path, "w", encoding="utf-8") as file:
    file.write(response.text)

with open(path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

In [86]:
#Obtendo a data da publicação
dt_publi = soup.find('meta', attrs={'name': 'cXenseParse:recs:publishtime'}).get('content')

#Converter a dt_publi para data
dt_publi = datetime.fromisoformat(dt_publi)

In [60]:
# Obtendo o título
titulo = soup.find('meta', property='og:title').get('content')
titulo

'Empresário do caso "Luso Brasileiro" acumula processos'

In [112]:
# Obtendo o corpo do texto da notícia
corpo = soup.find_all('div', class_='bloco-de-materia tw-paragrafo flex flex-col m-0 p-2 px-0')
text_corpo =""
for paragrafo in corpo:
    text_corpo += paragrafo.text.strip()



In [79]:
# Obtendo o subtítulo da notícia
subtitulo = soup.find('h3').text.strip()
subtitulo

'Rui Denardin foi condenado por desvio de verbas da Sudam, com outros réus, a devolver R$ 4,7 milhões, em 2008, que hoje atualizado pelo IPCA seria em torno de R$ 11.5 milhões.'

In [77]:
# Não há identificação do autor da notícia

In [113]:
dados = {
    "url": url,
    "data_requisicao": dt_requisição.isoformat(),
    "data_publicacao": dt_publi.isoformat(),
    "titulo": titulo,
    "subtitulo": subtitulo,
    "corpo": text_corpo
}

In [116]:
#Exportando em JSON
path = r"C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\resultado_exercicio_6.json"
with open(path, "w", encoding="utf-8") as file:
    json.dump(dados, file, ensure_ascii=False, indent=4)

# Exercício 7: 

#### Repita a questão anterior utilizando um site de notícias de sua escolha.

In [117]:
#Obtendo o HTML da notícia do G1

url = 'https://g1.globo.com/sp/sorocaba-jundiai/noticia/2024/09/19/especialista-explica-como-identificar-envenenamento-em-pets-e-o-que-fazer-em-casos-de-intoxicacao.ghtml'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                    '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

response = requests.get(url, headers=headers)
dt_requisição = datetime.now(pytz.utc)

path = r"C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\noticiaG1.html"
with open(path, "w", encoding="utf-8") as file:
    file.write(response.text)

with open(path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

In [120]:
#Obtendo a data da publicação
dt_publi = soup.find('meta', itemprop="datePublished").get('content')

#Converter a dt_publi para data
dt_publi = datetime.fromisoformat(dt_publi)

In [121]:
# Obtendo o título
titulo = soup.find('h1', class_='content-head__title').text.strip()
titulo

'Especialista explica como identificar envenenamento em pets e o que fazer em casos de intoxicação'

In [123]:
# Obtendo o corpo do texto da notícia
corpo = soup.find_all('p', class_='content-text__container')
text_corpo =""
for paragrafo in corpo:
    text_corpo += paragrafo.text.strip()


'Identificar o envenenamento em um pet pode ser um desafio para diversos tutores. Além disso, ver o sofrimento do animal pode causar desespero e, consequentemente, piorar as chances de entender a situação e agir de forma rápida e consciente.📲 Participe do canal do g1 Sorocaba e Jundiaí no WhatsAppO envenenamento em um pet pode ser causado por inseticidas ou outros venenos para combater pragas, mas também por produtos de limpeza e alimentos que possuímos em casa.Segundo a médica veterinária e professora da Universidade de Sorocaba (Uniso), Bianca Belline, quadros de intoxicação em cães e gatos são frequentes na rotina médica de emergência, sendo geralmente agudos e graves e exigem uma rápida intervenção, além de tratamentos adequados para reduzir o risco de morte do pet.“A menos que se presencie o momento em que o animal lambeu, ingeriu ou inalou uma substância, é difícil saber a causa da intoxicação. Intoxicações e envenenamentos podem ser acidentais e intencionais, por diversos agente

In [124]:
# Obtendo o subtítulo da notícia
subtitulo = soup.find('h2').text.strip()
subtitulo

'Envenenamento em um pet pode ser causado por inseticidas ou outros venenos para combater pragas, mas também por produtos de limpeza e alimentos que possuímos em casa.'

In [126]:
# Obtendo o autor
autor = soup.find('a', class_='multi_signatures').text.strip()
autor

'Gabriela Almeida'

In [128]:
dados = {
    "url": url,
    "data_requisicao": dt_requisição.isoformat(),
    "data_publicacao": dt_publi.isoformat(),
    "titulo": titulo,
    "subtitulo": subtitulo,
    "corpo": text_corpo,
    "autor": autor
}

In [129]:
#Exportando em JSON
path = r"C:\Users\RodrigoPintoMesquita\Documents\GitHub\Faculdade\Notebooks Trabalhos 2024H2\DR2_TP3\Dados\resultado_exercicio_7.json"
with open(path, "w", encoding="utf-8") as file:
    json.dump(dados, file, ensure_ascii=False, indent=4)

# Exercício 8: 
#### Examine o site https://difusoranews.com/ e descubra o padrão de URL para paginação que ele aceita (dica: considere tentativa-e-erro). Então, utilize-o para obter uma lista de links de notícias requisitando as cinco primeiras páginas e raspando os links através de um único seletor de CSS aplicado via `BeautifulSoup`.

In [172]:
dados = ""

for i in range(1,6):
    url = f'https://difusoranews.com/bem-estar/page/{i}/'
    response = requests.get(url)
    dados += response.text

In [177]:
soup = BeautifulSoup(dados, 'html.parser')
links = soup.select('h3.text-xl a')

for link in links:
    print(link.get('href'))


https://difusoranews.com/bem-estar/fogos-de-artificio-saiba-como-o-barulho-afeta-o-seu-pet/
https://difusoranews.com/bem-estar/oleaginosas-os-beneficios-das-nuts-as-proteinas-do-bem/
https://difusoranews.com/bem-estar/campanha-janeiro-roxo-combate-a-hanseniase/
https://difusoranews.com/bem-estar/dormir-tarde-da-noite-favorece-o-ganho-de-peso-diz-pesquisa/
https://difusoranews.com/bem-estar/ministerio-da-saude-distribui-novo-medicamento-contra-o-virus-da-aids/
https://difusoranews.com/bem-estar/rotulos-de-medicamentos-deverao-ter-alerta-de-substancias-consideradas-como-doping/
https://difusoranews.com/bem-estar/pomadas-modeladoras-saiba-quais-sao-os-cuidados-e-produtos-autorizados-pela-anvisa/
https://difusoranews.com/bem-estar/metas-para-um-novo-ano-como-cumprir-a-lista-de-promessas/
https://difusoranews.com/bem-estar/janeiro-branco-confira-6-dicas-para-cuidar-da-saude-mental/
https://difusoranews.com/bem-estar/o-que-nao-pode-comer-na-virada-do-ano-veja-dicas-de-cardapio/
https://difus

# Exercício 9:

#### Repita a questão anterior utilizando um site de notícias de sua escolha.

In [185]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                    '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

dados = ""

for i in range(1,6):
    url = f'https://g1.globo.com/busca/?q=not%C3%ADcias&page={i}'
    response = requests.get(url, headers=headers)
    dados += response.text

soup = BeautifulSoup(dados, 'html.parser')
links = soup.select('div.widget--info__text-container a')

for link in links:
    link_ajustado = 'https:' + str(link.get('href'))
    print(link_ajustado)


https://g1.globo.com/busca/click?q=not%C3%ADcias&p=1&r=1726853290222&u=https%3A%2F%2Fgloboplay.globo.com%2Fv%2F12938617%2F&syn=False&key=29265bf95957537f247a6eeaee191b45
https://g1.globo.com/busca/click?q=not%C3%ADcias&p=2&r=1726853290223&u=https%3A%2F%2Fg1.globo.com%2Fsc%2Fsanta-catarina%2Fnoticia%2F2023%2F10%2F06%2Fnsc-tv-informacoes-adicionais-dos-telejornais-bom-dia-sc-jornal-do-almoco-e-nsc-noticias.ghtml&syn=False&key=f9f4b3ffe844e1a872b3a940a9eaf339
https://g1.globo.com/busca/click?q=not%C3%ADcias&p=3&r=1726853290225&u=https%3A%2F%2Fg1.globo.com%2Fro%2Frondonia%2Fplaylist%2Fnoticias-de-rondonia.ghtml&syn=False&key=a03807add70aa6dab18c0a304c78a159
https://g1.globo.com/busca/click?q=not%C3%ADcias&p=4&r=1726853290226&u=https%3A%2F%2Fgloboplay.globo.com%2Fv%2F12937009%2F&syn=False&key=98f849e06a72be020d5b82bcfcf2ecf6
https://g1.globo.com/busca/click?q=not%C3%ADcias&p=5&r=1726853290228&u=https%3A%2F%2Fgloboplay.globo.com%2Fv%2F12935186%2F&syn=False&key=a15433ba03bc9342b7bdfac4b1026f9

# Exercício 10: 

#### Consulte o arquivo `robots.txt` do site https://tecnoblog.net/ para encontrar o link do `sitemap_index` e obtenha a partir dele a lista de todos os sitemaps de notícias (filtrando apenas os de notícias).

In [191]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get('https://tecnoblog.net/sitemap_index.xml', headers=headers)
soup = BeautifulSoup(response.text, 'xml')

In [195]:
lista_noticias = []

for sitemap in soup.find_all('sitemap'):
    links = sitemap.find('loc').text
    if 'noticias' in links:  
        lista_noticias.append(links)

lista_noticias

['https://tecnoblog.net/noticias-sitemap.xml',
 'https://tecnoblog.net/noticias-sitemap2.xml',
 'https://tecnoblog.net/noticias-sitemap3.xml',
 'https://tecnoblog.net/noticias-sitemap4.xml',
 'https://tecnoblog.net/noticias-sitemap5.xml',
 'https://tecnoblog.net/noticias-sitemap6.xml',
 'https://tecnoblog.net/noticias-sitemap7.xml',
 'https://tecnoblog.net/noticias-sitemap8.xml',
 'https://tecnoblog.net/noticias-sitemap9.xml',
 'https://tecnoblog.net/noticias-sitemap10.xml',
 'https://tecnoblog.net/noticias-sitemap11.xml',
 'https://tecnoblog.net/noticias-sitemap12.xml',
 'https://tecnoblog.net/noticias-sitemap13.xml',
 'https://tecnoblog.net/noticias-sitemap14.xml',
 'https://tecnoblog.net/noticias-sitemap15.xml',
 'https://tecnoblog.net/noticias-sitemap16.xml',
 'https://tecnoblog.net/noticias-sitemap17.xml',
 'https://tecnoblog.net/noticias-sitemap18.xml',
 'https://tecnoblog.net/noticias-sitemap19.xml',
 'https://tecnoblog.net/noticias-sitemap20.xml',
 'https://tecnoblog.net/notici

# Exercício 11: 

#### Escolha um dos sites de exemplo em https://toscrape.com/ para criar um projeto no Scrapy que obtenha os principais dados disponíveis, salvando em CSV ou JSON conforme a conveniência em relação aos dados do site escolhido.

In [ ]:
# Abaixo colei o código do Scrapy, contido no arquivo books.py
# O resultado pode ser conferido no arquivo "DR2_TP3\books_scraper\books.csv"

import scrapy

class BooksSpider(scrapy.Spider):
    name = "books"
    start_urls = ['https://books.toscrape.com/']

    def parse(self, response):
        for book in response.css('article.product_pod'):
            yield {
                'Titulo': book.css('h3 a::attr(title)').get(),
                'Preço': book.css('p.price_color::text').get(),
                'Disponibilidade': book.css('p.instock.availability i::attr(class)').get(),
                'Avaliação': book.css('p.star-rating::attr(class)').get(),
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)
            

# Exercício 12: 

#### Escolha um dos sites utilizados nas questões 6, 7, 8 e 9 para montar um projeto no Scrapy que abarque tanto o Crawling quanto o Scraping, a fim de rodá-lo tal como na questão anterior.

In [ ]:
# Abaixo colei o código do Scrapy, contido no arquivo news.py
# O resultado pode ser conferido no arquivo "DR2_TP3\news_scraper\news.csv"

import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"
    start_urls = [f'https://difusoranews.com/bem-estar/page/{i}/' for i in range(1, 6)]

    def parse(self, response):
        for link in response.css('h3.text-xl'):
            yield {
                 'titulo': link.css('a::attr(alt)').get(),
                 'link': link.css('a::attr(href)').get(),
                 }